# Глава 31. Дополнительные возможности классов

## Расширение встроенных типов

### Расширение типов встраиванием

Пример на основе самостоятельного класса, который управляет встроенным в него объектом списка

In [1]:
# setwrapper.py

class Set:
    def __init__(self, value=[]):   # Конструктор
        self.data = []              # Управляет списком
        self.concat(value)
        
    def intersect(self, other): # other – любая последовательность
        res = []                # self – подразумеваемый объект
        for x in self.data:     
            if x in other:
                res.append(x)   # Выбрать общие элементы
        return Set(res)         # Вернуть новый экземпляр Set
    
    def union(self, other):     # other – любая последовательность
        res = self.data[:]      # Копировать список
        for x in other:         # Добавить элементы из other
            if not x in res:
                res.append(x)
        return Set(res) 
    
    def concat(self, value):    # Аргумент value: список, Set...
        for x in value:         # Удалить дубликаты
            if not x in self.data:
                self.data.append(x)
                
    def __len__(self):
        return len(self.data)            # len(self)
    
    def __getitem__(self, key):
        return self.data[key]            # self[i]
    
    def __and__(self, other):
        return self.intersect(other)     # self & other
    
    def __or__(self, other):
        return self.union(other)         # self | other
    
    def __repr__(self):
        return 'Set:' + repr(self.data)  # Вывод 

In [2]:
x = Set([1, 3, 5, 7])
print(x.union(Set([1, 4, 7])))

Set:[1, 3, 5, 7, 4]


In [3]:
print(x | Set([1, 4, 6]))

Set:[1, 3, 5, 7, 4, 6]


### Расширение типов наследованием

>Все встроенные типы данных можно наследовать

Пример списка с индексированием с 1

In [4]:
class Mylist(list):
    def __getitem__(self, offset):
        print(f'indexing {self} at {offset}')
        return list.__getitem__(self, offset - 1)
    
if __name__ == '__main__':
    print(list('abc'))
    x = Mylist('abc')   # __init__ наследуется из списка
    print(x)            # __repr__ наследуется из списка (но str же?)
    print(x[1])         # Mylist.__getitem__
    print(x[3])         # изменяет поведение суперкласса
    
    x.append('spam')    # методы, унаследованные от list
    print(x)
    x.reverse()
    print(x)

['a', 'b', 'c']
['a', 'b', 'c']
indexing ['a', 'b', 'c'] at 1
a
indexing ['a', 'b', 'c'] at 3
c
['a', 'b', 'c', 'spam']
['spam', 'c', 'b', 'a']


Пример адаптирования списка добавлением методов и операторов, используемых для работы с множествами

In [5]:
# setsubclass.py
class Set(list):
    def __init__(self, value = []): # Конструктор
        list.__init__([])           # Адаптирует список
        self.concat(value) # Копировать изменяемый аргумент по умолчанию

    def intersect(self, other):  # other – любая последовательность
        res = []                 # self – подразумеваемый объект
        for x in self:
            if x in other:       # Выбрать общие элементы
                res.append(x)
        return Set(res)          # Вернуть новый экземпляр Set
    
    def union(self, other):      # other – любая последовательность
        res = Set(self)          # Копировать меня и мой список
        res.concat(other)
        return res 
    
    def concat(self, value):     # аргумент value: list, Set...
        for x in value:          # Удалить дубликаты
            if not x in self:
                self.append(x)
                
    def __and__(self, other): return self.intersect(other)
    
    def __or__(self, other): return self.union(other)
    
    def __repr__(self): return 'Set:' + list.__repr__(self)
    
    if __name__ == '__main__':
        x = Set([1,3,5,7])
        y = Set([2,1,4,5,6])
        print(x, y, len(x))
        print(x.intersect(y), y.union(x))
        print(x & y, x | y)

Set:[1, 3, 5, 7] Set:[2, 1, 4, 5, 6] 4
Set:[1, 5] Set:[2, 1, 4, 5, 6, 3, 7]
Set:[1, 5] Set:[1, 3, 5, 7, 2, 4, 6]


## Классы "нового стиля"

Разделение на "классические" и классы "нового стиля" осталась только в Python 2:
* В Python 3.0 все классы автоматически относятся к категории классов «нового стиля», независимо от того, наследуют ли они явно класс `object ` или нет. **Все классы наследуют `object`, явно или неявно, и все объекты являются экземплярами класса `object`**.


* В Python  2.6 и  в более ранних версиях классы должны явно наследовать класс `object` (или другой встроенный тип), чтобы считаться классами «нового стиля» и получить в свое распоряжение все особенности классов нового стиля.

### Отличия классов "нового стиля" (для Python 2)

* **Классы и типы были объединены** Классы теперь являются типами, а типы – классами. Фактически эти два термина стали синонимами. Вызов встроенной функции `type(I)` теперь возвращает класс, из которого был получен экземпляр; обычно тот, что указан в  атрибуте `I.__class__`, а  не обобщенный тип «instance». Кроме того, сами классы являются экземплярами класса `type`, который можно наследовать в подклассах для изменения процедуры создания классов, и все классы (а, следовательно, и типы) наследуют класс `object`.


* **Порядок поиска в дереве наследования** Принятая ромбоидальная схема множественного наследования немного изменила порядок поиска – грубо говоря, в этой схеме поиск сначала производится в ширину и только потом в высоту.


* **Извлечение атрибутов встроенными операциями** Встроенные операции больше не используют методы `__getattr__` и `__getattribute__` для неявного извлечения атрибутов. Это означает, что данные методы не вызываются для получения ссылок на методы перегрузки операторов с именами вида `__X__` – поиск таких имен начинается с класса, а не с экземпляра.


* **Новые особенности** Классы нового стиля приобрели ряд новых особенностей, включая слоты, свойства, дескрипторы и новый метод `__getattribute__`. В большинстве своем они предназначены для использования разработчиками, создающими инструментальные средства.

### Изменения в модели типов

С принятием модели классов нового стиля различия между типами и классами полностью исчезли. Теперь классы также являются типами: сами классы являются экземплярами класса `type`, а типами экземпляров классов являются их классы.

Фактически между встроенными типами, такими как списки и строки, и пользовательскими типами, определенными в виде классов, нет никаких
отличий. Именно поэтому имеется возможность наследовать встроенные типы
в своих классах.

С появлением классов нового стиля типом экземпляра класса является сам
класс, из которого он был создан, то есть классы являются обычными пользовательскими типами данных, – типом экземпляра является его класс, а пользовательские классы имеют тот же тип, что и встроенные объекты типов. Теперь классы тоже имеют атрибут `__class__`, потому что они являются экземплярами класса `type`:

In [6]:
class C: pass

I = C()
type(I)  # типом экземпляра является его класс

__main__.C

In [7]:
type(C)  # классы - это типы, а типы - это классы

type

In [8]:
C.__class__

type

In [9]:
type([1, 2, 3])  # классы и встроенные типы ничем не отличаются

list

In [10]:
type(list)

type

In [11]:
list.__class__

type

С технической точки зрения каждый класс создается из *метакласса* - класса с именет `type` или его подкласса, адаптирующего или управляющего процедурой создания классов.

### Значимость для операций проверки типа

В Python 3 типы экземпляров классов можно сравнивать непосредственно, точно так же, как сравниваются встроенные объекты типов.

In [12]:
class C: pass
class D: pass

c = C()
d = D()

type(c) == type(D)  # сравниваются классы экземпляров

False

In [13]:
type(c), type(d)

(__main__.C, __main__.D)

In [14]:
c.__class__, d.__class__

(__main__.C, __main__.D)

In [15]:
c1, c2 = C(), C()
type(c1) == type(c2)

True

В Python 2 так нельзя, т.к. все экземпляры в этой модели имеют один и тот же тип "instance".

### Все объекты наследуют класс `object`

Все классы наследуют класс `object`, явно или неявно. Т.е. каждый объект наследует встроенный класс `object`.

In [16]:
class C: pass

X = C()
type(X)  # теперь типом является класс экземпляра

__main__.C

In [17]:
type(C)  # типом класса является класс type

type

Однако также верно, что экземпляры и классы наследуют встроенный класс `object`, поскольку он явно или неявно является суперклассом любого класса

In [18]:
isinstance(X, object)

True

In [19]:
isinstance(C, object)

True

То же самое и для встроенных типов

In [20]:
type('spam')

str

In [21]:
type(str)

type

In [22]:
isinstance('spam', object)

True

In [23]:
isinstance(str, object)

True

>Фактически сам класс `type` наследует класс `object`, а класс `object` наследует класс `type`, даже при том, что оба они являются совершенно различными объектами, - циклическая связь, венчающая объектную модель и вытекающая из того факта, что типы являются классами, которые генерируют другие классы:

In [24]:
type(type)  # Все классы – это типы, и наоборот

type

In [25]:
type(object)

type

In [26]:
isinstance(type, object) # Все классы наследуют object, даже класс type

True

In [27]:
isinstance(object, type) # Типы создают классы, и type является классом

True

In [28]:
type is object

False

### Ромбоидальное наследование

В новой модели поиск в дереве наследования сначала производится в ширину – интерпретатор сначала просматривает все суперклассы, стоящие правее того, где поиск уже произведен, и только потом начинает подъем всеми возможными путями к общему суперклассу. Другими словами, поиск выполняется по уровням дерева наследования. В действительности интерпретатор использует немного более сложный алгоритм поиска, чем описано здесь, но этого упрощенного представления вполне достаточно для большинства программистов.

Пример

In [29]:
class A:
    attr = 1
    
class B(A):
    pass

class C(A):
    attr = 2
    
class D(B, C):  # сначала поиск дойдет до С, потом до А
    pass

x = D()
x.attr

2

### Явное разрешение конфликтов имен

Если вам требуется более полное управление процедурой поиска, вы всегда можете произвести выбор желаемого атрибута из любого места в дереве, выполнив присваивание или как-то иначе обозначив его там, где может возникнуть смешение классов:

In [30]:
class A:
    attr = 1
    
class B(A):
    pass

class C(A):
    attr = 2
    
class D(B, C):  # выбрать A.attr, выше
    attr = B.attr

x = D()
x.attr

1

Здесь мы явно выбирает аттрибуты или методы, выполняя присваивание именам, находящимся ниже в дереве. Мы могли бы просто вызвать метод желаемого класса явно – на практике этот подход, возможно, является более общепринятым, в особенности при работе с конструкторами:
```
class D(B, C):
    def meth(self):   # переопределяется ниже
        ...
        C.meth(self)  # вызовом выбрать метод класса C
```

### Пределы влияния изменений в порядке поиска

Кроме того, следует отметить, что даже если вы не будете применять ромбоидальную схему наследования в  своих классах, все равно суперкласс `object` в версии 3.0 всегда будет находиться выше любого класса, вследствие чего любой случай множественного наследования будет соответствовать ромбоидальной схеме.

То есть в новой модели класс `object` автоматически играет ту же самую роль, какую играет класс A в только что рассмотренном примере. Отсюда следует, что новые правила поиска не только изменили логическую семантику, но и  оптимизировали производительность, так как исключают возможность посещения одного и того же класса более одного раза.


Не менее важно то обстоятельство, что суперкласс `object` в новой модели предоставляет методы по умолчанию, реализующие различные встроенные операции, включая методы вывода `__str__` и `__repr__`.

In [31]:
dir(object)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

## Другие расширения в классах нового стиля

### Слоты экземпляров

Присваивая список имен атрибутов в  виде строк **специальному атрибуту `__slots__` класса**, в  классах нового стиля можно ограничить множество разрешенных атрибутов для экземпляров класса и оптимизировать использование памяти и производительность.

>**Только имена, перечисленные в списке `__slots__`, смогут использоваться как атрибуты экземпляра.**

Однако, как и в случае с любыми именами, прежде чем получить доступ к атрибутам экземпляра, им должны быть присвоены значения, даже если они перечислены в списке `__slots__`.

In [32]:
class limiter:
    __slots__ = ['age', 'name', 'job']

x = limiter()
x.age  # присваивание должно быть выполнено раньше использования

AttributeError: age

In [33]:
x.age = 40
x.age

40

In [34]:
x.ape = 1000  # недопустимое имя: отсутствует в списке __slots__

AttributeError: 'limiter' object has no attribute 'ape'

Слоты  – это своего рода нарушение динамической природы языка Python, которая диктует, что операция присваивания может создавать любые имена. Однако предполагается, что эта особенность поможет ликвидировать ошибки,
обусловленные простыми «опечатками» (обнаруживается попытка присваивания атрибутам, отсутствующим в списке `'__slots__'`), и обеспечит некоторую оптимизацию.

Выделение памяти для словаря с именами атрибутов в каждом экземпляре может оказаться слишком дорогим удовольствием, когда требуется создать большое количество экземпляров, каждый из которых обладает небольшим числом атрибутов. Для экономии пространства в памяти и повышения производительности (получающийся выигрыш в значительной степени
зависит от самой программы) атрибуты, перечисленные в слотах, сохраняются не в словаре, а в виде последовательности, что обеспечивает более высокую скорость их поиска.

### Слоты и обобщенные инструменты

Фактически некоторые экземпляры со слотами вообще могут не иметь атрибут словаря `__dict__`, что может сделать некоторые метапрограммы намного более сложными (включая некоторые из тех, что представлены в этой книге).

Обобщенные инструменты, которые получают списки атрибутов или обращаются к атрибутам, используя имена в виде строк, например, должны использовать более универсальные механизмы, чем атрибут `__dict__`. К таким механизмам можно отнести встроенные функции `getattr`, `setattr` и `dir`, способные отыскивать атрибуты в обоих хранилищах, `__dict__` и `__slots__`. В некоторых случаях для полноты картины может потребоваться проверить оба источника атрибутов.

Например, экземпляры классов, где используются слоты, обычно не имеют атрибут словаря `__dict__`  – вместо него пространство для атрибутов в экземпляре выделяется с применением дескрипторов класса, которые будут рассматриваться в главе 37. Только имена, перечисленные в списке `__slots__`, смогут использоваться как атрибуты экземпляра, однако значения этих атрибутов могут извлекаться и изменяться обычными способами.

>**По умолчанию наличие `__slots__` означает отсутствие `__dict__`**

In [35]:
class C:
    __slots__ = ['a', 'b']  # по умолчанию наличие __slots__
                            # означает отсутствие __dict__
        
X = C()
X.a = 1
X.a

1

In [36]:
X.__dict__

AttributeError: 'C' object has no attribute '__dict__'

In [37]:
getattr(X, 'a')

1

In [38]:
setattr(X, 'b', 2)  # однако ф-ии getattr() и setattr()
X.b                 # по-прежнему работают

2

In [39]:
# и dir() также отыскивает атрибуты в слотах
'a' in dir(X), 'b' in dir(X)

(True, True)

В отсутствие словаря с  пространством имен невозможно присвоить значения атрибутам экземпляра, имена которых отсутствуют в списке слотов:

In [40]:
class D:
    __slots__ = ['a', 'b']
    
    def __init__(self):  # невозможно добавить новый атрибут
        self.d = 4       # когда отсутствует атрибут __dict__
        
X = D()

AttributeError: 'D' object has no attribute 'd'

>Однако возможность добавлять новые атрибуты все-таки существует – для этого **необходимо включить имя `__dict__` в список `__slots__`, разрешив тем самым создать словарь с пространством имен**. В этом случае действовать будут оба механизма хранения имен, однако обобщенные инструменты, такие как `getattr`, будут воспринимать их, как единое множество атрибутов

In [41]:
class D:
    __slots__ = ['a', 'b', '__dict__']
    c = 3  # атрибуты действуют как обычно (__dict__ в __slots__)
    
    def __init__(self):  # имя d будет добавлено в __dict__
        self.d = 4       # а не в __slots__
        
X = D()
X.d

4

In [42]:
X.__dict__

{'d': 4}

In [43]:
X.__slots__

['a', 'b', '__dict__']

In [44]:
X.c

3

In [45]:
X.a

AttributeError: a

In [46]:
X.a = 1

In [47]:
list(map(lambda x: getattr(X, x), ['a', 'c', 'd']))

[1, 3, 4]

### Несколько суперклассов со списками `__slots__`

Обратите внимание, что в этой реализации просматривается содержимое атрибута `__slots__` только самого нижнего в дереве класса, наследуемого экземпляром. Если в  дереве имеется несколько классов, обладающих собственными атрибутами `__slots__`, универсальные инструменты должны иначе подходить к  получению списка атрибутов (например, рассматривать имена слотов, как атрибуты классов, а не экземпляров).

Объявления слотов могут присутствовать сразу в нескольких классах в дереве, но они имеют дополнительные ограничения, которые будет слишком сложно объяснить, пока вы еще не знаете, что слоты реализованы в виде дескрипторов на уровне класса (эту особенность мы детально будем рассматривать в последней части книги):

* Если подкласс наследует суперкласс, который не имеет атрибута `__slots__`, атрибут `__dict__` суперкласса будет доступен всегда, что делает бессмысленным использование атрибута `__slots__` в подклассе.


* Если класс определяет слот с тем же именем, что и суперкласс, версия имени, объявленная в суперклассе, будет доступна только при непосредственном обращении к дескриптору в суперклассе.


* Поскольку объявление `__slots__` имеет значение только для класса, в котором оно присутствует, подклассы автоматически получат атрибут `__dict__`, если не определят свой атрибут `__slots__`.

В общем для получения списка атрибутов экземпляра при использовании слотов в нескольких классах может потребоваться:

* подъем по дереву классов вручную,


* использование функции `dir` или подход, при котором имена слотов рассматриваются, как совершенно отдельная категория имен:

In [48]:
class E:
    __slots__ = ['c', 'd']  # суперкласс имеет слоты
    
class D(E):
    __slots__ = ['a', '__dict__']  # его подкласс также имеет слоты
    
X = D()
X.a = 1; X.b = 2; X.c = 3  # Экземпляр объединяет слоты в себе
X.a, X.c

(1, 3)

In [49]:
E.__slots__  # но в классах слоты не объединяются

['c', 'd']

In [50]:
D.__slots__

['a', '__dict__']

In [51]:
X.__slots__  # экземпляр наследует __slots__ ближайшего класса

['a', '__dict__']

In [52]:
X.__dict__  # и имеет собственный атрибут __dict__

{'b': 2}

In [53]:
dir(X)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 'a',
 'b',
 'c',
 'd']

### Свойства класса (`property`)

Механизм, известный как **свойства, обеспечивает в классах нового стиля еще один способ определения методов, вызываемых автоматически при обращении или присваивании атрибутам экземпляра**. Эта особенность во многих случаях представляет собой **альтернативу** методам перегрузки операторов `__getattr__` и `__setattr__`, которые мы рассматривали в главе 29.

Свойства обладают тем же эффектом, что и эти два метода, только в этом случае выполняется вызов метода даже при простом обращении к атрибуту, что бывает полезно для атрибутов, значения которых вычисляются динамически.

Проще говоря, **свойства  – это тип объектов, который присваивается именам атрибутов класса**. Они создаются вызовом **встроенной функции `property`**, которой передаются три метода (обработчики операций чтения, присваивания и удаления), и строкой документирования – если в каком-либо аргументе передается значение `None`, следовательно, эта операция не поддерживается.

Определение свойств обычно производится на верхнем уровне в  инструкции `class` (например, `name = property(...)`).

Пример с `__getattr__`

In [54]:
class classic:
    def __getattr__(self, name):
        if name == 'age':
            return 40
        else:
            raise AttributeError
            
x = classic()
x.age  # вызовет метод __getattr__

40

In [55]:
x.name  # вызовет метод __getattr__

AttributeError: 

То же самое с `property`

In [56]:
# ещё раз - __getattr__ используется при доступе к непределённому атрибуту
# если присвоить атрибут и попытаться получить к нему доступ, то AttributeError не будет
x.b = 42
x.b

42

In [57]:
class newprops:
    def getage(self):
        return 40
    
    age = property(getage, None, None, None)  # get, set, del, docs
    
x = newprops()
x.age  # вызовет метод getage

40

In [58]:
x.name  # нормальная операция извлечения

AttributeError: 'newprops' object has no attribute 'name'

В некоторых случаях свойства могут быть менее сложными и работать быстрее, чем при использовании традиционных подходов. Например, когда добавляется поддержка операции присваивания атрибуту, свойства становятся более привлекательными – программный код выглядит компактнее и в операцию присваивания не вовлекаются дополнительные вызовы методов, если не требуется производить дополнительных вычислений:

In [59]:
class newprops:
    def getage(self):
        return 40
    
    def setage(self, value):
        print('set age:', value)
        self._age = value
        
    age = property(getage, setage, None, None)  # get, set, del, docs
    
x = newprops()
x.age  # вызовет метод getage

40

In [60]:
x.age = 42  # вызовет метод setage

set age: 42


In [61]:
x._age  # нормальная операция извлечения, нет вызова getage

42

In [62]:
x.age  # используется getage

40

In [ ]:
x.job = 'trainer'  # нормальная операция извлечения, нет вызова setage
x.job  # нормальная операция извлечения, нет вызова getage

Для этого примера свойства обладают неоспоримым преимуществом. Однако в некоторых приложениях методы `__getattr__` и `__setattr__` по-прежнему могут быть востребованы для обеспечения более динамичных или универсальных
интерфейсов, чем можно реализовать с помощью свойств.

Например, во многих случаях невозможно заранее определить набор поддерживаемых атрибутов, которые могут даже не существовать вообще в каком-либо виде на момент написания класса (например, при делегировании ссылок на произвольные методы в обернутых/встроенных объектах).

### Метод `__getattribute__` и дескрипторы

Метод `__getattribute__` позволяет классам перехватывать **все** попытки обращения к атрибутам, а не только к неопределенным (как метод `__getattr__`). Кроме того, этот метод более сложен в обращении, чем `__getattr__`, из-за более высокой вероятности зациклвания.

В дополнение к свойствам и методам перегрузки операторов в Python поддерживается понятие **дескрипторов атрибутов** - классов, с методами `__get__` и `__set__`, которые присваиваются атрибутам классов. Они наследуются экземплярами и перехватывают попытки доступа к определенным атрибутам. Дескрипторы представляют собой, в некотором смысле, более обобщенную форму свойств.

Фактически свойства - это упрощенный вариант определения дескрипторов специфического типа, основанных на вызовах функций, управляющих доступом к атрибутам. Кроме того, дескрипторы используются для реализации слотов.

### Метаклассы

Метаклассы - подклассы объекта `type`, которые реализуют операции создания классов. Они обеспечивают отличную возможность управления объектами классов и их расширения.

## Статические методы и методы класса

**Статические методы** работают почти так же, как обычные функции, только расположенные внутри класса, а **методы класса** получают сам класс вместо экземпляра.

Чтобы сделать возможными эти режимы работы методов, внутри класса должны вызываться специальные встроенные функции **`staticmethod`** и **`classmethod`** или использоваться декораторы.

В Python 3 методы, которым не передается ссылка на экземпляр и которые вызываются только через имя класса, не требуют объявления с помощью функции `staticmethod`, но такое объявление обязательно для методов, которые предполагается вызывать через экземпляры.

### Зачем нужны специальные методы?

Иногда в программах бывает необходимо организовать обработку данных, связанных с классами, а не с экземплярами. Например, следить за числом экземпляров класса или вести список всех экземпляров класса, находящихся в настоящий момент в памяти. Такого рода информация связана с классами и  должна обрабатываться на уровне класса, а не экземпляров. То есть такая информация обычно сохраняется в самом классе и обрабатывается, независимо от наличия экземпляров класса.

Для решения таких задач часто бывает достаточно простых функций, определения которых находятся за пределами классов. Такие функции могут обращаться к атрибутам класса через его имя – им требуется доступ только к данным класса и  никогда  – к  экземплярам. Однако, чтобы теснее связать такой программный код с классом и обеспечить возможность его адаптации с помощью механизма наследования, будет лучше помещать такого рода функции внутрь
самого класса. Для этого нам и нужны методы класса, которые не ожидают получить аргумент `self` с экземпляром.

В языке Python для этих целей поддерживаются **статические методы – простые функции без аргумента `self`, вложенные в  определение класса и  предназначенные для работы с  атрибутами класса, а  не экземпляра**.

Статические методы никогда автоматически не получают ссылку `self` на экземпляр, независимо от того, вызываются они через имя класса или через экземпляр. Такие методы обычно используются для обработки информации, имеющей отношение ко всем экземплярам, а не для реализации поведения экземпляров.

Кроме того, в языке Python поддерживается также понятие **методов класса**. На практике методы класса используются реже, и в **первом аргументе им автоматически передается объект класса, независимо от того, вызываются они через имя класса или через экземпляр**. Такие методы могут получить доступ к данным класса через аргумент `self`, даже когда они вызываются относительно экземпляра. Обычные методы (которые формально называются методами экземпляра) при вызове получают подразумеваемый экземпляр, а статические методы и методы класса – нет.

### Статические методы

В Python 2 операция извлечения метода по имени класса возвращает *несвязанный метод*, при вызове которого требуется вручную передавать экземпляр

В Python 3 операция извлечения метода по имени класса возвращает **простую функцию**, которой не требуется передавать ссылку на экземпляр.

Другими словами, в Python 2.6 методам всегда необходимо передавать экземпляр, независимо от того, вызываются они через имя класса или через экземпляр. В  Python  3.0, напротив, экземпляр требуется передавать методам, только если они ожидают получить его, – методы без аргумента `self` могут вызываться через имя класса без передачи им ссылки на экземпляр. То есть в версии 3.0 допускается объявлять простые функции внутри класса, при условии, что они не ожидают получить и им не будет передаваться аргумент со ссылкой на экземпляр.

В результате:
* В Python  2.6 мы всегда должны объявлять метод как статический, чтобы иметь возможность вызывать его без передачи ссылки на экземпляр, независимо от того, вызываются он через имя класса или через экземпляр.


* В Python 3.0 от нас не требуется объявлять метод, как статический, если он будет вызываться только через имя класса, но мы обязаны объявлять его статическим, если он может вызываться через экземпляр.

В качестве примера предположим, что необходимо использовать атрибуты класса для подсчета числа экземпляров, созданных из класса. В следующем файле `spam.py` представлена первая попытка – класс содержит счетчик в виде
атрибута класса, конструктор, который наращивает счетчик при создании нового экземпляра, и метод, который выводит значение счетчика. Не забывайте, что атрибуты класса совместно используются всеми экземплярами. Поэтому
наличие счетчика непосредственно в объекте класса гарантирует, что он будет хранить число всех экземпляров:

In [63]:
class Spam:
    numInstances = 0
    
    def __init__(self):
        Spam.numInstances = Spam.numInstances + 1
        
    def printNumInstances():
        print('Number of instances created: ', Spam.numInstances)

Метод `printNumInstances` предназначен для обработки данных класса, а  не экземпляров – эти данные являются общими для всех экземпляров. Вследствие этого нам необходима возможность вызывать его, не передавая ссылку на экземпляр.

In [64]:
a, b, c = Spam(), Spam(), Spam()
Spam.printNumInstances()

Number of instances created:  3


In [65]:
a.printNumInstances()  # вызывается Spam.printNumInstances(a)

TypeError: printNumInstances() takes 0 positional arguments but 1 was given

In [66]:
type(Spam.printNumInstances)  # статический метод - это функция

function

### Альтернативы статическим методам

Кроме возможности пометить метод, как специальный, существуют и другие приемы, которые можно попробовать. Если для доступа к  атрибутам класса требуется вызывать функции, которые не принимают ссылку на экземпляр, самая простая мысль, которая приходит в  голову, – сделать метод обычной функцией, а не методом класса.

До появления статических методов в Python такой способ был единственным.

### Использование статических методов и методов класса

Классы со статическими методами и с методами класса не требуют передачи экземпляра класса в виде аргумента. Чтобы определить такие методы, в классах необходимо вызывать встроенные функции `staticmethod` и `classmethod`. Обе функции помечают объект функции как специальный, т.е. 

* в случае **`staticmethod` - как не требующий передачи экземпляра**,


* в случае **`classmethod` - как требующий передачи класса**.

Например:

In [67]:
class Methods:
    def imeth (self, x):  # обычный метод экземпляра
        print(self, x)
        
    def smeth(x):  # статический метод: экземпляр не передается
        print(x)
        
    def cmeth(cls, x):  # метод класса: получает класс, но не экз-р
        print(cls, x)
        
    smeth = staticmethod(smeth)  # сделать smeth статическим
    cmeth = classmethod(cmeth)   # сделать cmeth методом класса

Две последние функциии просто **переприсваивают** имена методов.

>С технической точки зрения Python поддерживает три разновидности методов:
>* методы экземпляра
>
>* статические методы
>
>* методы класса

**Методы экземпляра** - это обычные (и используемые по умолчанию) методы, которые мы видели в этой книге. Для воздействия на объект экземпляра всегда следует вызывать методы экземпляра.

In [68]:
obj = Methods()  # создать экземпляр

obj.imeth(1)  # обычный вызов, через экземпляр
              # будет преобразован в вызов imeth(obj, 1)

<__main__.Methods object at 0x7fe9fc774470> 1


In [69]:
Methods.imeth(obj, 2)  # обычный вызов, через класс
                       # экземпляр передается явно

<__main__.Methods object at 0x7fe9fc774470> 2


**Статические методы** вызываются без аргумента с экземпляром. В отличие от простых функций, за пределами класса их имена ограничены областью видимости класса, в котором они определяются и могут отыскиваться механизмом наследования. Применение встроенной функции `staticmethod` обеспечивает возможность вызывать такие методы через экземпляр **в версии 3.0, вызов через имя класса возможен и без применения `staticmethod`, но вызов через экземпляр - нет**.

In [70]:
Methods.smeth(3)  # вызов статического метода, через имя класса
                  # экземпляр не передается и не ожидается

3


In [71]:
obj.smeth(4)  # вызов статического метода через экземпляр
              # экземпляр не передается

4


**Методы класса** похожи на статические, но интерпретатор автоматически передает методам класса сам класс (а не экземпляр) в первом аргументе, независимо от того, вызываются они через имя класса или через экземпляр:

In [72]:
Methods.cmeth(5)  # вызов метода класса, через имя класса
                  # будет преобразован в вызов cmeth(Methods, 5)

<class '__main__.Methods'> 5


In [73]:
obj.cmeth(6)  # вызов метода класса через экземпляр
              # будет преобразован в вызов cmeth(Methods, 6)

<class '__main__.Methods'> 6


### Подсчет количества экземпляров с помощью статических методов

In [74]:
class Spam:           # Для доступа к данным класса используется
    numInstances = 0  # статический метод
    
    def __init__(self):
        Spam.numInstances += 1
        
    def printNumInstances():
        print('Number of instances:', Spam.numInstances)
    
    printNumInstances = staticmethod(printNumInstances)

In [75]:
a, b, c = Spam(), Spam(), Spam()
Spam.printNumInstances()  # вызывается, как простая функция

Number of instances: 3


In [76]:
a.printNumInstances()  # аргумент с экземпляром не передается, ошибки не возникает

Number of instances: 3


По сравнению с  простым перемещением `printNumInstances` за пределы класса, как описывалось ранее, эта версия требует дополнительный вызов функции `staticmethod`. При этом здесь область видимости имени функции ограничена классом (имя не будет вступать в конфликт с другими именами в модуле), программный код перемещен туда, где он используется (внутрь инструкции `class`), и подклассы получают возможность **адаптировать статический метод наследованием** – этот подход более удобен, чем импортирование функций из файлов, в  которых находятся определения суперклассов. Это иллюстрирует следующий подкласс и листинг нового интерактивного сеанса:

In [77]:
class Sub(Spam):
    def printNumInstances():    # Переопределяет статический метод
        print('Extra stuff...') # который вызывает оригинал
        Spam.printNumInstances()
    printNumInstances = staticmethod(printNumInstances)
    
a, b = Sub(), Sub()
a.printNumInstances()  # вызов через экземпляр подкласса, печатает все экземпляры класса Spam

Extra stuff...
Number of instances: 5


In [78]:
Sub.printNumInstances()  # вызов через имя подкласса

Extra stuff...
Number of instances: 5


In [79]:
Spam.printNumInstances()

Number of instances: 5


### Подсчет количества экземпляров с помощью методов класса

Аналогичные действия можно реализовать с помощью **метода класса**

In [80]:
class Spam:
    numInstances = 0
    
    def __init__(self):
        Spam.numInstances += 1
        
    def printNumInstances(cls):
        print('Number of instances:', cls.numInstances)
        
    printNumInstances = classmethod(printNumInstances)

In [81]:
a, b = Spam(), Spam()
a.printNumInstances()  # в первом аргументе передается класс

Number of instances: 2


In [82]:
Spam.printNumInstances()  # также в первом аргументе передается класс

Number of instances: 2


>Однако, используя методы класса, имейте в  виду, что они принимают класс, самый близкий к  объекту вызова. Это влечет за собой ряд важных последствий, который оказывают влияние на попытки изменить данные класса через переданный методу класс.

Например, если мы определим подкласс, адаптирующий предыдущую версию метода `Spam.printNumInstances` так, чтобы он дополнительно выводил свой аргумент `cls`, и запустим новый сеанс:

In [83]:
class Spam:
    numInstances = 0  # Отслеживает количество экземпляров

    def __init__(self):
        Spam.numInstances += 1
        
    def printNumInstances(cls):
        print('Number of instances:', cls.numInstances, cls)
        
    printNumInstances = classmethod(printNumInstances)

    
class Sub(Spam):
    def printNumInstances(cls):       # Переопределяет метод класса
        print('Extra stuff...', cls)  # Но вызывает оригинал
        Spam.printNumInstances()
        
    printNumInstances = classmethod(printNumInstances)

    
class Other(Spam): pass  # Наследует метод класса

всякий раз, когда будет вызываться метод класса, интерпретатор будет передавать ему самый близкий класс, даже для подклассов, не имеющих собственной реализации метода класса:

In [84]:
x, y = Sub(), Sub()
x.printNumInstances()  # вызов через экземпляр подкласса

Extra stuff... <class '__main__.Sub'>
Number of instances: 2 <class '__main__.Spam'>


In [85]:
Sub.printNumInstances()  # вызов через сам подкласс

Extra stuff... <class '__main__.Sub'>
Number of instances: 2 <class '__main__.Spam'>


В первом случае здесь метод класса вызывается через экземпляр подкласса `Sub`, и интерпретатор передает методу ближайший к экземпляру класс `Sub`. В данном случае никаких проблем не возникает, так как версия метода, переопределенная в классе `Sub`, явно вызывает оригинальную версию метода в суперклассе `Spam`, при этом метод суперкласса принимает класс `Spam` в первом аргументе.

Но посмотрите, что произойдет в случае обращения к объекту, который просто наследует метод класса:

In [86]:
z = Other()
z.printNumInstances()

Number of instances: 3 <class '__main__.Other'>


Здесь в последнем вызове методу класса `Spam` передается класс `Other`. В данном случае метод работает потому, что он всего лишь извлекает значение счетчика, который обнаруживает в классе `Spam`, благодаря механизму наследования.

Однако если бы этот метод попытался присвоить новое значение атрибуту класса, он изменил бы атрибут класса `Other`, а не `Spam`! В данном конкретном случае, вероятно, было бы лучше жестко указать имя класса, в котором производится изменение данных, чем полагаться на передаваемый аргумент класса.

### Подсчет экземпляров для каждого класса с помощью методов класса

Фактически методы класса всегда получают ближайший класс в дереве наследования, поэтому:

* Применение статических методов, в которых явно указывается имя класса, может оказаться более удачным решением для обработки данных класса.


* Методы классов лучше подходят для обработки данных, которые могут отличаться для каждого конкретного класса в иерархии.

Например, для реализации счетчиков экземпляров каждого класса в  отдельности лучше подошли бы методы класса. В  следующем примере суперкласс определяет метод класса, управляющий информацией о  состоянии, которая отличается для разных классов в  дереве,  – подобно тому, как методы экземпляра управляют информацией о состоянии экземпляров:

In [91]:
class Spam:
    numInstances = 0
    
    def count(cls): # Счетчик экземпляров для каждого отдельного класса
        cls.numInstances += 1 # cls – ближайший к экземпляру класс
        
    def __init__(self):  # Передаст self.__class__ для подсчета
        self.count()
    
    count = classmethod(count)

    
class Sub(Spam):
    numInstances = 0
    
    def __init__(self):      # Переопределяет __init__
        Spam.__init__(self)

    
class Other(Spam):           # Наследует __init__
    numInstances = 0

    
class Another(Spam):  # нет своего атрибута numInstances
    pass

In [88]:
x = Spam()
y1, y2 = Sub(), Sub()
z1, z2, z3 = Other(), Other(), Other()

x.numInstances, y1.numInstances, z1.numInstances

(1, 2, 3)

In [89]:
Spam.numInstances, Sub.numInstances, Other.numInstances

(1, 2, 3)

In [92]:
x = Spam()
y1, y2 = Sub(), Sub()
z1, z2, z3 = Other(), Other(), Other()
w1, w2, w3, w4 = Another(), Another(), Another(), Another()

x.numInstances, y1.numInstances, z1.numInstances, w1.numInstances  # у w1 не 4, а 5

(1, 2, 3, 5)

In [93]:
Spam.numInstances, Another.numInstances

(1, 5)

## Декораторы и метаклассы: часть 1

**Декораторы функций** обеспечивают способ определения специальных режимов работы функций, обертывая их дополнительным слоем логики, реализованной в виде других функций.

Декораторы функций представляют собой более универсальные инструменты: их удобно использовать для добавления самой разной логики не только к статическим методам, но и к любым другим функциям.

Например, их можно использовать для расширения функций программным кодом, выполняющим регистрацию вызовов этих функций, проверяющим типы передаваемых аргументов в процессе отладки и так далее. В некоторой степени декораторы функций напоминают шаблон проектирования делегирования, исследованный нами в главе 30, но их главная цель состоит в том, чтобы расширять определенные функции или методы, а не весь интерфейс объекта.

### Основы декораторов функций

Синтаксически декоратор функции - это разновидность объявления функции времени выполнения. Декоратор функции записывается в строке, непосредственно перед строкой с инструкцией `def`, которая определяет функцию или метод, и состоит из символа **`@`, за которым следует то, что называется метафункцией**, - функция (или другой вызываемый объект), которая управляет другой функцией.

Статические методы, например, могут быть оформлены в виде декораторов:

```
class C:
    @staticmethod  # синтаксис декорирования
    def meth():
        ...
```

С технической точки зрения, это объявление имеет тот же эффект, что и фрагмент ниже:

```
class C:
    def meth():
        ...
        
    meth = staticmethod(meth)  # повторное присваивание имени
```

Результат, возвращаемый функцией-декоратором, повторно присваивается имени метода. Декоратор может возвращать объекты любого типа, поэтому данный прием позволяет декоратору вставлять дополнительный уровень логики, который будет запускаться при каждом вызове. Декоратор функции может возвращать как оригинальную функцию, так и новый объект, в котором хранится оригинальная функция, переданная декоратору, которая будет вызываться косвенно после того, как будет выполнен дополнительный слой логики.

### Первый пример декоратора функций

В Python уже имеется несколько удобных встроенных функций, которые можно использовать как декораторы, но при этом мы также можем писать свои собственные декораторы.

В следующем примере метод `__call__` используется в определении класса, который сохраняет декорируемую функцию в экземпляре и перехватывает вызовы по оригинальному имени. А т.к. это класс, кроме всего прочего в нем имеется возможность хранить информацию о состоянии (счетчик произведенных вызовов).

In [94]:
class tracer:
    def __init__(self, func):
        self.calls = 0
        self.func = func
        
    def __call__(self, *args):
        self.calls += 1
        print(f'call {self.calls} to {self.func.__name__}')
        self.func(*args)
        
@tracer             # то же, что и spam = tracer(spam)
def spam(a, b, c):  # обертывает spam в объект-декоратор
    print(a, b, c)

In [95]:
spam(1, 2, 3)        # в действительности вызывается объект-обертка
spam('a', 'b', 'c')  # т.е. выз-ся метод __call__ в классе
spam(4, 5, 6)        # метод __call__ выпол-т доп. действия
                     # и вызывает оригинальную функцию

call 1 to spam
1 2 3
call 2 to spam
a b c
call 3 to spam
4 5 6


Данный декоратор действует, как обычная функция, принимающая позиционные аргументы, но он не возвращает результат вызова декорируемой функции, не обрабатывает именованные аргументы и не может декорировать методы классов (при декорировании методов метод `__call__` мог бы передавать только экземпляр класса `tracer`).

Существует множество способов декорирования функций, включая вложенные инструкции `def`, - некоторые из них лучше подходят для декорирования методов, чем способ, представленный здесь.

### Декораторы классов и метаклассы

**Декораторы классов** похожи на декораторы функций, но они запускаются после инструкции `class`, чтобы повторно присвоить имя класса вызываемому объекту. Они могут использоваться для изменения классов сразу после их создания или добавлять дополнительный слой логики уже после создания экземпляров.

При применении декоратора к классу программный код вида:

```
def decorator(cls): ...

@decorator
class C: ...
```

отображается в следующий эквивалент:

```
def decorator(cls): ...

class C: ...

C = decorator(C)
```

Декоратор класса может расширить функциональность самого класса или вернуть объект, который будет перехватывать последующие попытки конструирования экземпляров.

Например, для подсчета экземпляров класса:

In [96]:
def count(cls):
    cls.numInstances = 0
    return cls  # возвращает класс, а не обертку

@count
class Spam: ...  # то же, что и Spam = count(Spam)
    
@count
class Sub(Spam): ...  # инструкция numInstances = 0 не нужна здесь
    
@count
class Other(Spam): ...

In [97]:
a = Spam()

In [98]:
a.numInstances  # имеется атрибут numInstances, просто логика подсчета экземпляров не прописана

0

**Метаклассы** представляют собой похожий инструмент на основе классов, область применения которого отчасти перекрывает область применения декораторов классов. Они представляют альтернативную модель управления созданием объектов классов за счет создания подклассов типа `type` и включения их в инструкцию `class`:

```
class Meta(type):
    def __new__(meta, classname, supers, classdict): ...
    
class C(metaclass=Meta): ...
```

Обычно метакласс переопределяет метод `__new__` или `__init__` класса `type`, с целью взять на себя управление созданием или инициализацией нового объекта класса. Как и при использовании декораторов классов, суть состоит в том, чтобы определить программный код, который будет вызываться автоматически на этапе создания классов. Оба способа позволяют расширять классы или возвращать произвольные объекты для его замены - протокол с практически неограниченными возможностями.

## Типичные проблемы при работе с классами

Большая часть типичных проблем, связанных с классами, сводится к проблемам, связанным с пространствами имен (особенно если учесть, что классы - это всего лишь пространства имен с некоторыми дополнительными особенностями).

### Изменение атрибутов класса может приводить к побочным эффектам

Теоретически классы (и экземпляры классов) относятся к категории **изменяемых объектов**. Это означает, что изменение класса или экземпляра может оказывать влияние на множественные ссылки на них. Изменяя атрибуты, следует от этом помнить.

Все экземпляры класса совместно используют одно и то же пространство имен класса, поэтому любые изменения на уровне класса будут отражаться на всех экземплярах, если последние не имеют собственных версий атрибутов класса.

In [99]:
class X:
    a = 1  # атрибут класса
    
I = X()
I.a  # унаследован экземпляром

1

In [100]:
X.a

1

Если атрибут класса изменяется динамически, за пределами инструкции `class`, то это приводит к одновременному изменению атрибута во всех объектах, наследующих его от класса.

In [101]:
X.a = 2  # может измениться только в классе X
I.a      # объект I тоже изменился

2

In [102]:
J = X()  # унаследует значение, установленное во время выполнения
J.a

2

### Модификация изменяемых атрибутов класса также может иметь побочные эффекты

Если атрибут класса ссылается на изменяемый объект, изменение этого объекта из любого экземпляра отразится сразу на всех экземплярах:

In [103]:
class C:
    shared = []           # атрибут класса
    
    def __init__(self):
        self.perobj = []  # атрибут экземпляра
        
x, y = C(), C()  # два экз-ра, неявно исп-т один и тот же атр. класса
y.shared, y.perobj

([], [])

In [104]:
x.shared.append('spam')  # окажет влияние на объект y также!

In [105]:
x.perobj.append('spam')  # изменит данные, принадл-ие только x
x.shared, x.perobj

(['spam'], ['spam'])

In [106]:
y.shared, y.perobj  # в y есть изменения, призв-ые через x

(['spam'], [])

In [107]:
C.shared  # сохраненный в классе и совместно исп-ый

['spam']

### Множественное наследование: порядок имеет значение

В случае использования множественного наследования порядок, в котором перечислены суперклассы в строке заголовка инструкции `class`, может иметь критическое значение. В ходе поиска интерпретатор просматривает суперклассы слева направо, в соответствии с порядком их следования в заголовке инструкции.

Если необходимо, чтобы одно имя наследовалось от одного суперкласса, а другое от другого, изменение порядка расположения в заголовке не поможет - мы должны вручную переопределить результат наследования, явно выполнив присваивание имени атрибута в подклассе:

In [108]:
class ListTree:
    def __str__(self): ...
        
    def other(self): ...
        
class Super:
    def __str__(self): ...
    def other(self): ...
        
        
class Sub(ListTree, Super): # Унаследует __str__ класса ListTree, так как он
                            # первый в списке
    other = Super.other # Явно выбирается версия атрибута из класса Super
    def __init__(self):
        ...
        
x = Sub() # Поиск сначала выполняется в Sub и только потом в ListTree/Super

Как правило, множественное наследование дает лучшие результаты, когда суперклассы являются максимально автономными, - поскольку они могут использоваться в разных контекстах, они не должны делать каких-либо предположений об именах, связанных с другими классами в дереве.

Псевдочастные атрибуты (`__X`) могут помочь в локализации имен, на владение которыми опирается класс, и ограничить вероятность появления конфликтов имен в суперклассах, которые вы добавляете в список наследуемых классов.

### Делегирование в версии 3.0: `__getattr__` и встроенные операции

Классы, использующие метод `__getattr__` для делегирования обернутым объектам операций обращения к атрибутам, будут терпеть неудачу в Python 3, если методы перегрузки операторов не будут переопределены в классе-обертке. В Python 3 обращения к методам перегрузки операторов производятся встроенными операциями неявно, минуя обычную схему выбора методов-обработчиков.

Например, метод `__str__`, используемый для вывода, никогда не вызывает `__getattr__`. Вместо этого в Python 3 интерпретатор пытается отыскать требуемые имена в классах, пропуская этап поиска в экземпляре. Чтобы решить эту проблему, подобные методы должны быть переопределены в классах-обертках.

### Многослойное обертывание

Если классы наслоены друг на друга слишком глубоко, программный код становится малопонятным  – возможно, вам придется изучить множество классов, чтобы выяснить, что делает единственная операция.

Не усложняйте решение задачи, если оно не является таковым. Обертывание программного кода несколькими слоями классов на грани непостижимости – всегда плохая идея. Абстракция – это основа полиморфизма и инкапсуляции, и  при грамотном использовании она может быть весьма эффективным инструментом. Однако вы упростите отладку и  сопровождение, если сделаете интерфейсы своих классов интуитивно понятными. Избегайте чрезмерной абстракции и сохраняйте иерархии своих классов короткими и плоскими, если не существует веских причин сделать иначе.